In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

### 1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.)

In [2]:
X, y = datasets.make_classification(n_features = 2, n_informative = 2, 
                                  n_classes = 2, n_redundant=0, 
                                  n_clusters_per_class=1, random_state=5)

In [3]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [4]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction

In [5]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [6]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [7]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [8]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5
    
    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [9]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, max_depth=5, depth=0, leafs=0):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0 or depth == max_depth:
        leafs += 1
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    depth+=1
    true_branch = build_tree(true_data, true_labels, max_depth=max_depth, depth=depth)
    false_branch = build_tree(false_data, false_labels, max_depth=max_depth, depth=depth)
    

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [11]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [12]:
train_data, test_data, train_labels, test_labels = train_test_split(X, 
                                                                    y, 
                                                                    test_size = 0.3,
                                                                    random_state = 1)

### 2. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [25]:
# Расчет дисперсии

def variance(values):
    val_mean = np.mean(values)
    squares = [(i-val_mean)**2 for i in values]
    return sum(squares) / len(values)

In [36]:
class Leaf:
    
    def __init__(self, data, values):
        self.data = data
        self.values = values
        self.prediction = self.predict()
        
    def predict(self):
        return np.mean(self.values)

In [81]:
# Расчет качества

def quality(left_values, right_values, current_variance):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_values.shape[0]) / (left_values.shape[0] + right_values.shape[0])
    return current_variance - p * variance(left_values) - (1 - p) * variance(right_values)

In [59]:
# Разбиение датасета в узле

def split(data, values, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_values = values[left]
    false_values = values[right]
        
    return true_data, false_data, true_values, false_values

In [76]:
# Нахождение наилучшего разбиения

def find_best_split(data, values):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5
    
    current_variance = variance(values)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_values, false_values = split(data, values, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_values, false_values, current_variance)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index
    return best_quality, best_t, best_index

In [83]:
# Построение дерева с помощью рекурсивной функции

def build_reg_tree(data, values, max_depth=5, depth=0):

    quality, t, index = find_best_split(data, values)
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0 or depth == max_depth:
        return Leaf(data, values)

    true_data, false_data, true_values, false_values = split(data, values, index, t)

    # Рекурсивно строим два поддерева
    depth+=1
    true_branch = build_tree(true_data, true_values, max_depth=max_depth, depth=depth)
    false_branch = build_tree(false_data, false_values, max_depth=max_depth, depth=depth)
    

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [78]:
X, y = datasets.make_regression(n_features = 2, n_informative = 2, 
                                random_state=5)
train_data, test_data, train_values, test_values = train_test_split(X, y, 
                                                                    test_size = 0.3,
                                                                    random_state = 1)

In [85]:
reg_tree = build_reg_tree(train_data, train_values)

In [99]:
reg_prediction = predict(test_data, reg_tree)
list(zip(reg_prediction, test_values))

[(-54.35634172577482, -55.67013163043917),
 (123.1031262020856, 150.66051504602567),
 (-54.35634172577482, -17.137616546106955),
 (-29.105630694331246, -43.41898493517199),
 (-10.772916465924025, -21.91407211672093),
 (-10.772916465924025, -12.571975864464301),
 (17.019366109004096, 25.98976096788553),
 (123.1031262020856, 121.6919120931539),
 (-10.772916465924025, 30.35614963220106),
 (-54.35634172577482, -22.263512937823407),
 (17.019366109004096, -5.3197115651654885),
 (35.95087900163848, 31.431729391091757),
 (-54.35634172577482, -81.39203045659491),
 (-29.105630694331246, -27.19301046785796),
 (37.4238776327042, 34.16535050582929),
 (123.1031262020856, 61.78979212314055),
 (35.95087900163848, 23.07697180143311),
 (-54.35634172577482, -45.449106527600236),
 (123.1031262020856, 139.74658291870088),
 (7.798014762375311, 6.538121998877713),
 (123.1031262020856, 102.45299265672081),
 (7.798014762375311, -29.315841176558145),
 (61.9558421220885, 66.14773594011402),
 (-10.772916465924025

In [100]:
from sklearn.metrics import mean_squared_error, r2_score

In [101]:
r2_score(test_values, reg_prediction)

0.8558102546515577

In [97]:
mean_squared_error(test_values, reg_prediction)

496.16389376803176

#### Получаем $R^2$ = 0.86 на тестовой выборке, дерево работает